<a href="https://colab.research.google.com/github/Arshpreet-Singh-1/ai-machine-learning-for-coders/blob/main/3_2_Horses_vs_Humans_ImageDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.Imports

In [1]:
import tensorflow as tf
print(tf.__version__)

2.8.2


# 2. Training and Validation Dataset

In [2]:
import urllib.request
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_url =  "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
urllib.request.urlretrieve(training_url, training_file_name)

training_dir  = 'horse-or-human/training/'
zip_ref = zipfile.ZipFile(training_file_name , 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

train_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size = (300,300),
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [3]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
urllib.request.urlretrieve(validation_url , validation_file_name)

validation_dir = 'horse-or-human/validation/'
zip_ref = zipfile.ZipFile(validation_file_name , 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (300,300),
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


# 3. Build and Compile Model

In [4]:
model = tf.keras.models.Sequential([
                                   tf.keras.layers.Conv2D(16 , (3,3) , activation = 'relu' , input_shape = (300,300,3) ),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(32 , (3,3) , activation = 'relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(64 , (3,3) , activation = 'relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(64 , (3,3) , activation = 'relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Conv2D(64 , (3,3) , activation = 'relu'),
                                   tf.keras.layers.MaxPooling2D(2,2),
                                   tf.keras.layers.Flatten(),
                                   tf.keras.layers.Dense(units = 512 , activation = 'relu'),
                                   tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
])

In [5]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(learning_rate = 0.001) , loss = 'binary_crossentropy' , metrics = ['accuracy'])

# 4. Fit Model 

In [6]:
model.fit( train_generator, epochs = 15 , validation_data = validation_generator)

Epoch 1/15
33/33 [==============================] - 30s 724ms/step - loss: 0.7254 - accuracy: 0.5190 - val_loss: 0.6770 - val_accuracy: 0.7461
Epoch 2/15
33/33 [==============================] - 24s 761ms/step - loss: 0.6351 - accuracy: 0.6631 - val_loss: 1.0361 - val_accuracy: 0.5273
Epoch 3/15
33/33 [==============================] - 24s 730ms/step - loss: 0.5246 - accuracy: 0.7371 - val_loss: 2.1044 - val_accuracy: 0.5977
Epoch 4/15
33/33 [==============================] - 24s 729ms/step - loss: 0.4341 - accuracy: 0.8043 - val_loss: 0.5947 - val_accuracy: 0.7305
Epoch 5/15
33/33 [==============================] - 24s 724ms/step - loss: 0.3876 - accuracy: 0.8510 - val_loss: 2.1691 - val_accuracy: 0.5703
Epoch 6/15
33/33 [==============================] - 24s 750ms/step - loss: 0.3059 - accuracy: 0.8832 - val_loss: 0.9646 - val_accuracy: 0.7422
Epoch 7/15
33/33 [==============================] - 24s 731ms/step - loss: 0.3093 - accuracy: 0.8919 - val_loss: 1.5584 - val_accuracy: 0.7656

# 5. Running the Model

In [7]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path , target_size = (300,300))
  x = image.img_to_array(img)
  x = np.expand_dims(x , axis = 0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a horse")
  else:
    print(fn + " is a human")